<a href="https://colab.research.google.com/github/Didier06/IA_FABLAB/blob/main/Yolov8_Minst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ======================================================
# 🧠 MNIST multi-chiffres avec YOLOv8m (Ultralytics)
# ======================================================

!pip install ultralytics torchvision tqdm pyyaml --quiet

from torchvision import datasets
from PIL import Image
import random, os, yaml
from tqdm import tqdm

In [ ]:
# ======================================================
# 1️⃣ Monter Google Drive pour sauvegarder le modèle
# ======================================================
from google.colab import drive
drive.mount('/content/drive')

save_dir = "/content/drive/MyDrive/MNIST_YOLOv8m"
os.makedirs(save_dir, exist_ok=True)
print(f"📁 Modèle sera sauvegardé dans : {save_dir}")

In [ ]:
# ======================================================
# 2️⃣ Génération du dataset multi-chiffres (3 à 10)
# ======================================================
mnist = datasets.MNIST(root="data", train=True, download=True)

os.makedirs("mnist_yolo/images/train", exist_ok=True)
os.makedirs("mnist_yolo/labels/train", exist_ok=True)
os.makedirs("mnist_yolo/images/val", exist_ok=True)
os.makedirs("mnist_yolo/labels/val", exist_ok=True)

def create_images(nb_images, split):
    for i in tqdm(range(nb_images), desc=f"Génération {split}"):
        img = Image.new("L", (320, 320), color=0)
        nb_digits = random.randint(3, 10)
        label_lines = []

        for _ in range(nb_digits):
            idx = random.randint(0, len(mnist) - 1)
            digit_img, label = mnist[idx]
            x = random.randint(0, 320 - 28)
            y = random.randint(0, 320 - 28)
            img.paste(digit_img, (x, y))

            xc = (x + 14) / 320
            yc = (y + 14) / 320
            w = 28 / 320
            h = 28 / 320
            label_lines.append(f"{label} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}")

        img.save(f"mnist_yolo/images/{split}/{i}.jpg")
        with open(f"mnist_yolo/labels/{split}/{i}.txt", "w") as f:
            f.write("\n".join(label_lines))

create_images(4000, "train")
create_images(1000, "val")

In [ ]:
# ======================================================
# 3️⃣ Fichier de configuration data.yaml
# ======================================================
import yaml

with open("mnist_yolo/data.yaml", "w") as f:
    yaml.dump({
        'path': '/content/mnist_yolo',  # dossier racine du dataset
        'train': 'images/train',
        'val': 'images/val',
        'nc': 10,
        'names': [str(i) for i in range(10)]
    }, f)

print("✅ Dataset YOLOv8 prêt")


In [ ]:
!nvidia-smi

In [ ]:
# ======================================================
# 🔁 Entraînement YOLOv8m avec vraie reprise Drive (même dossier, même epoch)
# ======================================================
import os
from ultralytics import YOLO
from google.colab import drive

# 1️⃣ Monter Google Drive
drive.mount('/content/drive')

# 2️⃣ Paramètres
drive_dir = '/content/drive/MyDrive/yolo_mnist_training'
run_name  = 'mnist_yolo_run'
data_yaml = '/content/mnist_yolo/data.yaml'  # chemin de ton YAML
base_model = 'yolov8m.pt'
img_size = 320
epochs_total = 50

# 3️⃣ Dossier de run complet
run_dir = f"{drive_dir}/{run_name}"
os.makedirs(run_dir, exist_ok=True)

# 4️⃣ Vérifie s’il y a un checkpoint existant
last_ckpt = os.path.join(run_dir, 'weights', 'last.pt')

if os.path.exists(last_ckpt):
    print(f"🔁 Reprise automatique depuis : {last_ckpt}")
    model = YOLO(last_ckpt)
    resume_mode = True
else:
    print("🆕 Nouveau départ avec yolov8m.pt")
    model = YOLO(base_model)
    resume_mode = False

# 5️⃣ Entraînement avec vraie reprise native
results = model.train(
    data=data_yaml,
    epochs=epochs_total,
    imgsz=img_size,
    project=drive_dir,     # <-- ton Drive comme projet
    name=run_name,         # <-- même nom de run (critique)
    resume=resume_mode,    # ✅ permet à YOLO de reprendre dans le même dossier
    save_period=2,         # ✅ sauvegarde tous les 5 epochs
    exist_ok=True,         # ✅ évite la création de _2, _3, _8
    device=0               # GPU unique (Colab)
)

print("✅ Entraînement terminé ou repris avec succès !")


In [ ]:
# ======================================================
# 5️⃣ Visualisation des courbes (loss et mAP)
# ======================================================
import pandas as pd
import matplotlib.pyplot as plt

# 🧭 Chemin vers ton dossier de run sur Google Drive
metrics_path = "/content/drive/MyDrive/yolo_mnist_training/mnist_yolo_run/results.csv"

# Lecture du fichier CSV
df = pd.read_csv(metrics_path)

# --- Visualisation ---
plt.figure(figsize=(12,5))

# 📉 Pertes (Loss)
plt.subplot(1,2,1)
plt.plot(df['epoch'], df['train/box_loss'], label='Box Loss')
plt.plot(df['epoch'], df['train/cls_loss'], label='Class Loss')
plt.plot(df['epoch'], df['train/dfl_loss'], label='DFL Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Courbes de perte (Loss)')
plt.legend()

# 📈 Précision (mAP)
plt.subplot(1,2,2)
plt.plot(df['epoch'], df['metrics/mAP50'], label='mAP@50')
plt.plot(df['epoch'], df['metrics/mAP50-95'], label='mAP@50-95')
plt.xlabel('Epoch')
plt.ylabel('mAP')
plt.title('Précision moyenne (mAP)')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# ======================================================
# 6️⃣ Test de prédiction (sur ton modèle sauvegardé sur Drive)
# ======================================================
from ultralytics import YOLO

# Chemin vers le modèle entraîné (best.pt sur ton Drive)
model_path = "/content/drive/MyDrive/yolo_mnist_training/mnist_yolo_run/weights/best.pt"

# Charger le modèle final
model = YOLO(model_path)

# Image de test
test_image = "mnist_yolo/images/val/5.jpg"

# Lancer la prédiction et sauvegarder le résultat
results = model.predict(source=test_image, save=True, imgsz=320)

# Afficher la prédiction directement dans Colab
results[0].show()
